<a href="https://colab.research.google.com/github/JLrumberger/TensorflowTryOuts/blob/master/2_Run_Model_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploy a Tensorflow Model on Google Cloud TPUs
## Transfer the model to a [tf.Estimator](https://www.tensorflow.org/guide/custom_estimators) model

The easiest way to deploy a tf model on TPUs is via the tf.Estimator wrapper tf.TPUEstimator. So the first step is to port the above ResNet model to a tf.Estimator and then port it from there to a TPUEstimator model.

In [0]:
import numpy as np
import tensorflow as tf

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def resblock(x, filters, is_training):
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    return x

def ResNet(x,is_training):
    filters = 64
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    x = tf.layers.conv2d(x, filters=filters, kernel_size=(3,3),padding='same')
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = tf.reshape(x, [-1, 7*7*filters])
    x = tf.layers.dense(x,512,activation='relu')
    out = tf.layers.dense(x,10)
    return out

def model_fn(features, labels, mode):
  # Specify the model
  logits = ResNet(x = features["x"],
                  is_training=(mode == tf.estimator.ModeKeys.TRAIN))

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    # update op for batch_norm layer
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Load training and eval datasets
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
  
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="/tmp/mnist_convnet_model")

# Set up logging for predictions
#tensors_to_log = {"probabilities": "softmax_tensor"}
tensors_to_log = {}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=1000)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=2000,
    hooks=[logging_hook])
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

## Migrate the model to a [TPUEstimator](https://www.tensorflow.org/api_docs/python/tf/contrib/tpu/TPUEstimator)
In order to run a TPUEstimator model on Colab TPUs, one needs to [set up a gcs bucket](https://cloud.google.com/storage/docs/creating-buckets) and authenticate with a google account to access the gcs bucket from within Colab. After that, the changes describes in 
[How to migrate a model](https://cloud.google.com/tpu/docs/tutorials/migrating-to-tpuestimator-api) need to be implemented on the estimator model.

In [0]:
import json
import os
import pprint
import tensorflow as tf
import time

use_tpu = True #@param {type:"boolean"}
bucket = 'gcolab' #@param {type:"string"}

assert bucket, 'Must specify an existing GCS bucket name'
print('Using bucket: {}'.format(bucket)) #gcolab

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

MODEL_DIR = 'gs://{}/{}'.format(bucket, time.strftime('tpuestimator/%Y-%m-%d-%H-%M-%S'))
print('Using model dir: {}'.format(MODEL_DIR))

from google.colab import auth
auth.authenticate_user()

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
  
  # Upload credentials to TPU.
  with tf.Session(TF_MASTER) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  pprint.pprint(session.list_devices())

In [0]:
# Load training and eval datasets
import numpy as np
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

params = {"x_train": {"x": train_data},
          "y_train": train_labels,
          "x_test" : {"x": eval_data},
          "y_test" : eval_labels,
          "iterations_per_loop": 1024, # after each loop the TPU passes back information to host CPU
          "train_batch_size":1024,
          "eval_batch_size":1024,
          "predict_batch_size":1024,
          "epochs": 80,
          "eval_steps":1024,
          "nb_MC_samples":100}


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


TPUEstimator requires all input functions to work with parameters. Furthermore TPUEstimator sets the batch_sizes as a key value pair in params for TRAIN, EVAL and PREDICT mode as specified in TPUEstimator.train_batch_size, .eval_batch_size, .predict_batch_size.

The below network uses MC dropout, so it is an extension of the original ResNet based on [Gal and Ghahramani, 2015](https://arxiv.org/pdf/1506.02142.pdf)

In [0]:
import numpy as np
import os
import tensorflow as tf

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def resblock(x, filters, is_training):
    # dropout always true, because it's MC dropout
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.dropout(x, rate=0.2, training=True)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.dropout(x, rate=0.2, training=True)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    return x

def ResNet(x,is_training):
    filters = 64
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    x = tf.layers.conv2d(x, filters=filters, kernel_size=(3,3),padding='same')
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = tf.reshape(x, [-1, 7*7*filters])
    x = tf.layers.dropout(x, rate=0.5, training=True)
    x = tf.layers.dense(x,512,activation='relu')
    out = tf.layers.dense(x,10)
    return out
  
def model_fn(features, labels, mode, params):
  # Specify the model
  logits = ResNet(x = features["x"],
                  is_training=(mode == tf.estimator.ModeKeys.TRAIN))

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    learning_rate = tf.train.exponential_decay(0.001,
          #FLAGS.learning_rate,
          tf.train.get_global_step(),
          decay_steps=100000,
          decay_rate=0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
    # update op for batch_norm layer
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss=loss,
            global_step=tf.train.get_global_step())
    return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  return tf.contrib.tpu.TPUEstimatorSpec(
      mode=mode, loss=loss, eval_metrics=(metric_fn,[labels,logits]))

  
def metric_fn(labels, logits):
    accuracy = tf.metrics.accuracy(
        labels=labels, predictions=tf.argmax(logits, axis=1))
    return {"accuracy": accuracy}

# Create model input functions
def train_input_fn(params):
  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((params["x_train"], params["y_train"]))
  # Shuffle, repeat, and batch the examples.
  dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.batch(params["batch_size"], drop_remainder=True)
  return dataset

def eval_input_fn(params):
  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((params["x_test"], params["y_test"]))
  # Shuffle, repeat, and batch the examples.
  dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.batch(params["batch_size"], drop_remainder=True)
  return dataset

def predict_input_fn(params):
  # generates MC dropout probabilities
  dataset = tf.data.Dataset.from_tensor_slices(params["x_test"])
  dataset = dataset.repeat(params["nb_MC_samples"])
  dataset = dataset.batch(params["batch_size"])
  return dataset
  
# Create the Estimator
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=MODEL_DIR, # google cloud services bucket
    session_config=tf.ConfigProto(
        allow_soft_placement=True, log_device_placement=True),
    tpu_config=tf.contrib.tpu.TPUConfig(params["iterations_per_loop"])
)

estimator = tf.contrib.tpu.TPUEstimator(
    model_fn=model_fn,
    use_tpu=True,
    train_batch_size=params["train_batch_size"],
    eval_batch_size=params["eval_batch_size"],
    predict_batch_size=params["predict_batch_size"],
    config=run_config,
    params=params
)

# Train the model

estimator.train(input_fn=train_input_fn, steps=params["epochs"]*params["train_batch_size"])

estimator.evaluate(input_fn= eval_input_fn, steps=params["eval_steps"])



INFO:tensorflow:Using config: {'_model_dir': 'gs://gcolab/tpuestimator/2019-01-21-17-01-07', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
log_device_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.42.1.186:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f57c35c65f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.42.1.186:8470', '_evaluation_master': b'grpc://10.42.1.186:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1024, num_

{'accuracy': 0.9877739, 'global_step': 122880, 'loss': 0.036972858}

In [0]:
predictions = list(estimator.predict(input_fn=predict_input_fn))
MC_probs = [p["probabilities"] for p in predictions]
MC_probs = np.array(MC_probs).reshape([params["nb_MC_samples"],10000,10])
predicted_probs = np.mean(MC_probs, axis=0)
predicted_std = np.std(MC_probs, axis=0)

from sklearn.metrics import accuracy_score
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
ev = one_hot(eval_labels,10)
accuracy_score(y_true=ev,y_pred=np.round(predicted_probs,0))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://gcolab/tpuestimator/2019-01-21-17-01-07/model.ckpt-122880
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 9 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of 

0.9898